In [ ]:
import numpy as np
import pandas
import string
import urllib
from bs4 import BeautifulSoup as bs
import datetime
import dateutil
from pprint import pprint
import os
import re
from datetime import datetime
print('ok')

In [ ]:
# Read dataset
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

### SUMMARY STATISTICS FOR FILENAME DATASET
print(len(music_df))
# 2,973 filenames

print('\n')
print(
    music_df
    .groupby('source')
    .size()
    .sort_values(ascending=False)
)
# Sources:
#DimeaDozen      1804
#Etree            564
#LosslessLegs     275
#TradersDen       330

print('\n')
print(
    music_df['filename'].
    groupby(music_df['filename'].
            apply(lambda x: x.lower().split('.')[-1])).
    size().
    sort_values(ascending=False)
)
# File extensions:
#flac    2798
#shn       92
#mp2       48
#mp3       31
#wav        1
#cue        1
#aud        1
#FLAC       1
print('ok')

In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
print('ok')

In [ ]:
# GET LIST OF OFFICIAL ABBREVIATIONS

pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
print(len(abbrev_df))

In [ ]:
debug = 1
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
 
# Initialize score to -9999
music_df['score'] = -9999

# Create outmusic_df as the place to move data to when a score has been assigned
thecolumns = music_df.columns
outmusic_df = pandas.DataFrame(data=None, columns=thecolumns) # Trick to copy schema but not data
outmusic_df.index.name = music_df.index.name

# All filenames have a period somewhere
#music_df[music_df['filename'].apply(lambda x: '.' not in x)]

# First, strip filename and convert to lowercase.
music_df['filename'] = music_df['filename'].apply(lambda x: x.lower().strip())

# Next, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])

#print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


# STEP 1: If there are no numbers at all, there's no date, so score as zero
if (debug > 0):
    print('Step 1: filter out filenames with no numbers, which cannot possibly be correct.')
music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score']][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


music_df['band'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()].lower().strip())

# Fix U2
music_df.loc[music_df['band'] == 'u', 'band'] = 'u2'

music_df['after_first_digit'] = music_df['remainder'].apply(lambda x: x[re.search('\d', x).start():len(x)].lower().strip())

music_df['full_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = ''
music_df.loc[music_df['full_date'].notnull(), 'mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[10+re.search('[\.\_\-\ ]*',x[10:]).end():])


music_df['parsed_date'] = pandas.to_datetime(np.nan)
music_df['yy_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df.loc[music_df['yy_date'].notnull(), 'mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[8+re.search('[\.\_\-\ ]*',x[8:]).end():])

fdy = re.compile('(19|20)\d\d[\ \-\_]+\d+[\ \-\_]+\d+')

music_df.loc[(music_df['full_date'].isnull()) & 
             (music_df['yy_date'].isnull()) &
             (music_df['parsed_date'].isnull()) &
             (music_df['after_first_digit'].apply(lambda x: re.search(fdy, x) != None)), 
         'parsed_date'
        ] = pandas.to_datetime(
    music_df['filename'][
        (music_df['full_date'].isnull()) & 
        (music_df['parsed_date'].isnull()) &
        (music_df['after_first_digit'].apply(lambda x: re.search(fdy, x) != None))
        ].apply(lambda x: x[re.search(fdy, x).start():re.search(fdy, x).end()])
    , errors='coerce'
)

In [ ]:

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)
            
#music_df['variant_abbrev'] = False
#music_df.loc[music_df['band'].isin(variant_abbrevs), 'variant_abbrev'] = True
#variant_abbrevs
#

In [ ]:

#music_df['remainder'][music_df['band'] == 'u2']

In [ ]:
# STEP 2: Band has canonical abbreviation and date is yyyy-mm-dd
if (debug > 0):
    print('\n')
    print('Step 2: Band has canonical abbreviation and date is yyyy-mm-dd.')


canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()

music_df['canonical_abbrev'] = False
music_df.loc[music_df['band'].isin(canonical_abbrevs), 'canonical_abbrev'] = True
    
music_df.loc[(music_df['canonical_abbrev']) & (music_df['full_date'].notnull()), 'score'] = 4
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'full_date', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['full_date'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# STEP 3: Band has canonical abbreviation and date is yy-mm-dd
if (debug > 0):
    print('\n')
    print('Step 3: Band has canonical abbreviation and date is yy-mm-dd.')
music_df.loc[(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull()), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source', 'filename', 'filetype', 'score', 'band', 'yy_date', 'mics_etc']][(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())])

if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
#music_df[(music_df['canonical_abbrev']) & (music_df['yy_date'].notnull())]


In [ ]:
debug = 1
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
 
# Initialize score to -9999
music_df['score'] = -9999

# Create outmusic_df as the place to move data to when a score has been assigned
thecolumns = music_df.columns
outmusic_df = pandas.DataFrame(data=None, columns=thecolumns) # Trick to copy schema but not data
outmusic_df.index.name = music_df.index.name

# All filenames have a period somewhere
#music_df[music_df['filename'].apply(lambda x: '.' not in x)]

# First, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])

#print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# STEP 1: If there are no numbers at all, there's no date, so score as zero
if (debug > 0):
    print('Step 1: filter out filenames with no numbers, which cannot possibly be correct.')
music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score']][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# STEP 2: DOES THE BAND ABBREVIATION MATCH AN OFFICIAL ABBREVIATION?

# STEP 2a: Parse out the band name and check against the abbreviations list
# first, parse to first number - that's where the date starts (usually, but not for U2 or 4 Non-Blondes)
music_df['band_maybe'] = music_df['remainder'].apply(lambda x: x.lower()[0:re.search('\d', x).start()])
music_df['abbrev_ok'] = False
music_df.loc[music_df['band_maybe'].isin(abbrev_df['abbrev'][abbrev_df['abbrev'] != '']), 'abbrev_ok'] = True
# abbrev_ok = True: 296

# Step 2b: If abbrev_ok, cut band name off front of remainder
music_df.loc[music_df['abbrev_ok'], 'remainder'] = music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[re.search('\d', x).start():])

# Step 2c: Look for proper datetime format of yyyy-mm-dd and assign score of 4
if (debug > 0):
    print('\n')
    print('Step 2c: Band abbreviation from official list plus date in correct yyyy-mm-dd format: score as 4.')
music_df['thedate'] = pandas.to_datetime(music_df[music_df['abbrev_ok']]['remainder'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 4])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2d: Format yy-mm-dd gets score of 3
if (debug > 0):
    print('\n')
    print('Step 2d: Band abbreviation from official list plus date in incorrect yy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]

if (debug > 0):    
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2e: Three clusters of numbers or xx, optionally with - or / separator, is a date.
if (debug > 0):
    print('\n')
    print('Step 2e: Band abbreviation from official list plus date in other incorrect format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['abbrev_ok']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['abbrev_ok']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 2f: No date provided but band abbrev OK, score as 3
if (debug > 0):
    print('\n')
    print('Step 2f: Band abbreviation from official list plus no date provided: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['abbrev_ok']), 'score'] = 3
music_df.loc[music_df['abbrev_ok'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['abbrev_ok'], 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])    
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))


# STEP 3: DOES THE BAND ABBREVIATION MATCH AN VARIANT ABBREVIATION? MODIFY AND REPEAT STEPS (a) through (f) above

# strip whitespace from band name and put as lowercase for easier processing
music_df['band_maybe'] = music_df['band_maybe'].apply(lambda x: x.lower().strip())

# Make a list of non-canonical abbreviations to check
otherabraw = abbrev_df['otherabbrev'][abbrev_df['otherabbrev'].apply(lambda x: x not in ['',' '])].drop_duplicates().values.tolist()

# Parse comma-separated values in non-canonical abbreviations list as separate possibilities; also strip whitespace
otherabbrevs = []
for x in otherabraw:
    if (',' in x):
        for y in x.split(','):
            otherabbrevs.append(y.strip())
    else:
        otherabbrevs.append(x.strip())

# If band name is a match to that non-canonical abbrev list, set abbrev_variant = True
music_df['abbrev_variant'] = False
music_df.loc[music_df['band_maybe'].isin(otherabbrevs), 'abbrev_variant'] = True
#print(len(music_df[music_df['abbrev_variant']]))
# abbrev_variant = True: 40

# Step 3b: If abbrev_variant, cut band name off front of remainder
music_df.loc[music_df['abbrev_variant'], 'remainder'] = music_df['remainder'][music_df['abbrev_variant']].apply(lambda x: x[re.search('\d', x).start():])

# Step 3c: Look for proper datetime format of yyyy-mm-dd; with the variant band name assign score of 3
if (debug > 0):
    print('\n')
    print('Step 3c: Band abbreviation from variant list plus date in correct yyyy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df[music_df['abbrev_variant']]['remainder'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 3d: Format yy-mm-dd plus variant band name gets score of 2
if (debug > 0):
    print('\n')
    print('Step 3d: Band abbreviation from variant list plus date in incorrect yy-mm-dd format: score as 2.')

music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['abbrev_variant']].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 2
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 3e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus variant band name, score as 2
if (debug > 0):
    print('\n')
    print('Step 3e: Band abbreviation from variant list plus date in other incorrect format: score as 2.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['abbrev_variant']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['abbrev_variant']) & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 2
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 3f: No date provided but band abbrev is a variant, score as 2
if (debug > 0):
    print('\n')
    print('Step 3f: Band abbreviation from variant list plus no date provided: score as 2.')

music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['abbrev_variant']), 'score'] = 2
music_df.loc[music_df['abbrev_variant'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['abbrev_variant'], 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 2]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 2])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

    
# Step 4: If they have the word "track" in the title, gets scored as zero (confirmed by eye)
#music_df['filename'][music_df['track_as_band']].sort_values().tolist()
if (debug > 0):
    print('\n')
    print('Step 4: The word "track" is present in the title: score as 0 (confirmed by eye as correct approach).')
music_df['track_as_band'] = False
music_df.loc[music_df['band_maybe'].apply(lambda x: 'track' in x), 'track_as_band'] = True
music_df.loc[music_df['track_as_band'], 'band_maybe'] = ''
music_df.loc[music_df['track_as_band'], 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[music_df['track_as_band'], 'mics_etc'] = music_df['remainder']
music_df.loc[(music_df['track_as_band']), 'score'] = 0
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 0]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 0])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    
    
# STEP 5a: Apply fix for U2
music_df.loc[music_df['band_maybe'].apply(lambda x: x.lower()) == 'u', 'band_maybe'] = 'U2'

# Step 5b: Fix remainder for U2
music_df.loc[music_df['band_maybe'] == 'U2', 'remainder'] = music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[re.search('u2[\ \-\_]*',x.lower()).end():len(x)])

#print('ok')
if (debug == 2):
    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'u'])
    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'U2'])

# Step 5c: band is U2 (no official abbrev) and date is yyyy-mm-dd, score as 4.
if (debug > 0):
    print('\n')
    print('Step 5c: Band is U2 plus date in correct yyyy-mm-dd format: score as 4.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x.lower()[0:10]), format='%Y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 4])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5d: Band is U2, format yy-mm-dd gets score of 3
if (debug > 0):
    print('\n')
    print('Step 5d: Band is U2 plus date in incorrect yy-mm-dd format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''

music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus band is U2, score as 3
if (debug > 0):
    print('\n')
    print('Step 5e: Band is U2 plus date in other incorrect format: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
music_df['mics_etc'] = music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 5f: No date provided but band is U2, score as 3
if (debug > 0):
    print('\n')
    print('Step 5f: Band is U2 plus no date provided: score as 3.')
music_df['thedate'] = ''
music_df['mics_etc'] = ''
music_df.loc[(music_df['band_maybe'] == 'U2'), 'score'] = 3
music_df.loc[(music_df['band_maybe'] == 'U2'), 'thedate'] = pandas.to_datetime(np.nan)
music_df.loc[(music_df['band_maybe'] == 'U2'), 'mics_etc'] = music_df['remainder']
outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
if (debug > 1):
    print('Processing filenames:')
    print(music_df['filename'][music_df['score'] == 3])
music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
if (debug > 0):
    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    

neworder = ['score', 'band_maybe', 'thedate', 'mics_etc', 'filetype', 'source', 'filename', 'id']
outmusic_df = outmusic_df[neworder]
outmusic_df = outmusic_df.rename(columns={'band_maybe': 'band'})

#outmusic_df.to_csv('outmusic.csv')
#print('File written')

print('\n')
print('Three rows from the output data, selected at random:')
outmusic_df.sample(3)

In [ ]:
# Band provided but not processed from abbrevs list
#print(len(music_df))

# I think these are trying to provide track number instead of band name
badformat = ['d', 't', 'a']

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms', 'ph']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-']


# Band names not in abbreviation list
notfound = ['loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']

vc = music_df['band_maybe'][music_df['band_maybe'] != ''].value_counts()
# If a band name appears 4 or fewer times, assume it's a misc band not found in the list.
for x in (vc[(vc<5)].index.sort_values().tolist()):
    if ((x not in notfound) and (x not in incorrect) and (x not in badformat)):
        notfound.append(x)
    

# STEP 6a: Used incorrect band name
music_df['remainder'][music_df['band_maybe'].apply(lambda x: x in incorrect)].apply(lambda x: x[0:19])


music_df.loc[music_df['abbrev_ok'], 'remainder'] = music_df['remainder'][music_df['abbrev_ok']].apply(lambda x: x[re.search('\d', x).start():])


# Step 6b: Set remainder by Parsing to first digit


#music_df.loc[music_df['band_maybe'].apply(lambda x: x.lower()) == 'u', 'band_maybe'] = 'U2'

# Step 5b: Fix remainder for U2
#music_df.loc[music_df['band_maybe'] == 'U2', 'remainder'] = music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[re.search('u2[\ \-\_]*',x.lower()).end():len(x)])

#print('ok')
#if (debug == 2):
#    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'u'])
#    print(music_df[['band_maybe', 'remainder']][music_df['band_maybe'] == 'U2'])

# Step 5c: band is U2 (no official abbrev) and date is yyyy-mm-dd, score as 4.
#if (debug > 0):
#    print('\n')
#    print('Step 5c: Band is U2 plus date in correct yyyy-mm-dd format: score as 4.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x.lower()[0:10]), format='%Y-%m-%d', errors='coerce')
#music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[10:])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 4
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 4]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 4])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5d: Band is U2, format yy-mm-dd gets score of 3
#if (debug > 0):
#    print('\n')
#    print('Step 5d: Band is U2 plus date in incorrect yy-mm-dd format: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''

#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][music_df['band_maybe'] == 'U2'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce')
#music_df['mics_etc'] = music_df['remainder'][music_df['thedate'].notnull()].apply(lambda x: x[8:])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))
    
# Step 5e: Three clusters of numbers or xx, optionally with - or / separator, is a date; plus band is U2, score as 3
#if (debug > 0):
#    print('\n')
#    print('Step 5e: Band is U2 plus date in other incorrect format: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df['thedate'] = pandas.to_datetime(music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x)).notnull())].apply(lambda x: x[0:re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+', x).end()]), errors='ignore')
#music_df['mics_etc'] = music_df['remainder'][(music_df['band_maybe'] == 'U2') & (music_df['remainder'].apply(lambda x: re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x)).notnull())].apply(lambda x: x[re.search('[\dxx]+[\-\/]*[\dxx]+[\-\/]*[\dxx]+\.*', x).end():len(x)])
#music_df.loc[music_df['thedate'].notnull(), 'score'] = 3
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))

# Step 5f: No date provided but band is U2, score as 3
#if (debug > 0):
#    print('\n')
#    print('Step 5f: Band is U2 plus no date provided: score as 3.')
#music_df['thedate'] = ''
#music_df['mics_etc'] = ''
#music_df.loc[(music_df['band_maybe'] == 'U2'), 'score'] = 3
#music_df.loc[(music_df['band_maybe'] == 'U2'), 'thedate'] = pandas.to_datetime(np.nan)
#outmusic_df = outmusic_df.append(music_df[['id', 'source','filename', 'filetype', 'score', 'band_maybe', 'abbrev_ok', 'thedate', 'mics_etc']][music_df['score'] == 3]).sort_values(by='score', ascending=False)
#if (debug > 1):
#    print('Processing filenames:')
#    print(music_df['filename'][music_df['score'] == 3])
#music_df = music_df.loc[~music_df.index.isin(outmusic_df.index)]
#if (debug > 0):
#    print('Filenames processed so far: {0:,.0f}'.format(len(outmusic_df)))
#    print('Filenames remaining: {0:,.0f}'.format(len(music_df)))    

#neworder = ['score', 'band_maybe', 'thedate', 'mics_etc', 'filetype', 'source', 'filename', 'id']
#outmusic_df = outmusic_df[neworder]
#outmusic_df = outmusic_df.rename(columns={'band_maybe': 'band'})
#
##outmusic_df.to_csv('outmusic.csv')
##print('File written')

#print('\n')
#print('Three rows from the output data, selected at random:')
#outmusic_df.sample(3)

#print(music_df[music_df['band_maybe'] != ''].groupby(['band_maybe']).size().sort_values(ascending=False))


#vc[(vc==1)].sort_index().index.tolist()
#music_df[music_df['band_maybe'] == 'dm']
#print(notfound)